In [1]:
from pandas import read_csv, DataFrame, Series, concat
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE
import matplotlib
import math

Load meta data

In [2]:
meta = read_csv('neurofibroma_meta_data_1.txt', sep='\t', index_col=0)

Load total RNA-Seq counts

In [3]:
df0 = read_csv('neurofibroma_total_counts.txt', sep='\t', index_col=0).T

RNA-Seq data for human Schwann cells (SRP094118)

In [4]:
df_schwann_1 = read_csv('normal_Schwann_SRP094118.txt', sep='\t', index_col=0)

RNA-Seq data for human Schwann cells (SRP212780)

In [5]:
S1 = read_csv('GSM3923266_Schwann.Cell.Human.1.counts.csv', sep=',', index_col=0)
S2 = read_csv('GSM3923267_Schwann.Cell.Human.2.counts.csv', sep=',', index_col=0)
S3 = read_csv('GSM3923268_Schwann.Cell.Human.3.counts.csv', sep=',', index_col=0)
S = DataFrame({'Schwann1': S1['Human_1'], 'Schwann2': S2['Human_2'], 'Schwann3': S3['Human_3']})
S.index.name = ''
df_schwann_2 = S.loc[sorted(S.index)].T

use only genes that are common in all 3 data sets

In [6]:
common_genes = sorted(set(df0.columns) & set(df_schwann_1.columns) & set(df_schwann_2.columns))

In [7]:
len(common_genes)

15484

Merge data sets and normalize together to account for read-depth differences

In [8]:
df2 = concat([df0[common_genes], df_schwann_1[common_genes], df_schwann_2[common_genes]])

Normalization using median-of-ratios method
$$s_j=\mbox{median}_{i:K_i^R\neq 0}\frac{K_{ij}}{K_i^R}$$ with $$K_i^R=\left(\prod_{j=1}^mK_{ij}\right)^{1/m}$$
This averages log counts per gene over all samples. Only use genes that don't have any zero counts.

In [9]:
df2_no_zeros = df2[df2!=0].dropna(axis=1)
KR = np.exp(np.log(df2_no_zeros).mean())
S = (df2_no_zeros / KR).median(axis=1) # sample-specific normalization factors
df2_normalized = (df2.T / S).T

Plot log2-transformed and normalized counts (+ a constant 0.1 to deal with zero counts) for a few samples   
As expected peaks of "switched-on" genes lie on top of each other because of the normalization, and exhibit approximately a log-normal distribution

In [10]:
matplotlib.rcParams['figure.figsize'] = (5, 5)
np.log2(df2_normalized+0.1).loc['pnNF95.12B'].hist(bins=50)
np.log2(df2_normalized+0.1).loc['2-032 Plexiform Neurofibroma'].hist(bins=50)
np.log2(df2_normalized+0.1).loc['Schwann1'].hist(bins=50, alpha=0.5)
np.log2(df2_normalized+0.1).loc['Expr Other1'].hist(bins=50, alpha=0.5)
plt.title('normalized log-transformed RNAseq counts (example)')

Text(0.5,1,'normalized log-transformed RNAseq counts (example)')

Perform PCA + tSNE

In [11]:
pca = PCA(n_components=10)
df2_pca = DataFrame(pca.fit_transform(np.log2(df2_normalized+0.1)), index=df2_normalized.index)
df2_tsne = DataFrame(TSNE(n_components=2).fit_transform(df2_pca), index=df2_pca.index)

... and visualize with overlaid tumor type

In [13]:
matplotlib.rcParams['figure.figsize'] = (10, 10)
for s in set(meta['tumorType']):
    I = meta[meta['tumorType']==s].index
    X = df2_tsne[:-8].loc[I]
    plt.scatter(X[0], X[1], s=10, label=s)
X = df2_tsne[-8:-3]
plt.scatter(X[0], X[1], s=10, label='Normal Schwann 1')
X = df2_tsne[-3:]
plt.scatter(X[0], X[1], s=10, label='Normal Schwann 2')
plt.legend()
plt.title('10dim PCA, tSNE (%s)' % 'tumorType')

Text(0.5,1,'10dim PCA, tSNE (tumorType)')

focus only on tumor types that derive from Schwann cells

In [14]:
focus_tumor_types = ['Neurofibroma', 'Cutaneous Neurofibroma', 'Plexiform Neurofibroma', 'Malignant Peripheral Nerve Sheath Tumor', 'Schwannoma']
normal_schwann1, normal_schwann2 = list(df2_normalized[-8:-3].index), list(df2_normalized[-3:].index)
focus_samples = set(meta[meta['tumorType'].isin(focus_tumor_types)].index)
focus_samples_ = [n for n in meta[meta['tumorType'].isin(focus_tumor_types)].index if n in set(df2_normalized.index)] + normal_schwann1 + normal_schwann2

In [15]:
df2_tsne_focus = df2_tsne.loc[focus_samples_]
df2_pca_focus = df2_pca.loc[focus_samples_]

In [16]:
for s in set(meta['tumorType']):
    I = meta[meta['tumorType']==s].index
    I = [t for t in I if t in focus_samples]
    X = df2_tsne_focus[:-8].loc[I]
    plt.scatter(X[0], X[1], s=10, label=s)
X = df2_tsne_focus[-8:-3]
plt.scatter(X[0], X[1], s=10, label='Normal Schwann 1')
X = df2_tsne_focus[-3:]
plt.scatter(X[0], X[1], s=10, label='Normal Schwann 2')
plt.legend()
plt.title('10dim PCA, tSNE (%s)' % 'tumorType')

Text(0.5,1,'10dim PCA, tSNE (tumorType)')